# Linear algebra basics

This notebook provides in introduction to essential elements of linear algebra needed to understand the dimensionality reduction algorithms performed by *flowTorch*.

## Multiplication
### Matrix-vector multiplication
### Matrix-matrix multiplication

## Special types of matrices
### Identity matrix
### Orthogonal matrix
### Orthonormal matrix
### Unitary matrix

## Eigen-decomposition
### ...
### Application to linear operators

## Singular value decomposition
### ...
### Low-rank approximation
### Least-squares approximation
### Connection so Eigen-decomposition